# Mining data on twitter with python.

Based on this article: https://marcobonzanini.com/2015/03/09/mining-twitter-data-with-python-part-2/

Requirements:
- TWEEPY 3.7
- nltk
- re
- json
- operator
- collections
- string
- vincent
- pandas

In [104]:
import tweepy
from tweepy import OAuthHandler

In [2]:
from tweepy import Stream
from tweepy.streaming import StreamListener

In [11]:
from nltk.tokenize import word_tokenize

In [24]:
import re, json

In [29]:
import operator 
from collections import Counter

In [31]:
from nltk.corpus import stopwords
import string

In [61]:
from nltk import bigrams

In [128]:
from collections import defaultdict

In [140]:
import vincent

In [152]:
import pandas as pd

## Part 1: Collecting data

In [91]:
with open('config.json') as f:
    credentials = json.load(f)

In [92]:
credentials

{'ACCESS_TOKEN': '720396530759385088-8bQxnzGDXXGHQB63eumuFYWnmL4nF6H',
 'ACCESS_TOKEN_SECRET': 'by2vprExn9FF4Sg2QJupvpZqhyiVYBr62FjB2VhkF0esv',
 'CONSUMER_KEY': 'RRZFSHS1Nw8xH4eRyBaftyFfh',
 'CONSUMER_SECRET': 'xdw3qoO1OmwZsb95CpDNpY4X7nYOwwgGeAPN2y6oiI9xP6Xkbx'}

In [105]:
consumer_key = credentials['CONSUMER_KEY'] 
consumer_secret = credentials['CONSUMER_SECRET']
access_token = credentials['ACCESS_TOKEN']
access_token_secret = credentials['ACCESS_TOKEN_SECRET']

In [106]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [107]:
public_tweets = api.home_timeline()

In [6]:
for tweet in public_tweets:
    print(tweet.text)

RT @ValetteClaude: Je veux des vidéos de vous entrain de chanter.
RT @ubereats_fr: La fille est Arya Stark de Winterfell. Et je commande pour ma maison ! 
#GameOfThrones en exclusivité sur @OCSTV. https://…
RT @Jarrynette34: Quand en soiree tu veux pas quon te pique ton verre 🤣 #VTEP https://t.co/ntoqXNXz7c
RT @CORNEVILLE: Ou va t'il chercher ces nouvelles  epreuves @Arthur_Officiel  #VTEP
RT @boytoygone: #VTEP excellent depuis le temps qu'il fallait des nouvelles épreuves top mDr 😅😅😅
RT @soldatguerrier1: Je Kiffe le 360 c'est la meilleure invention @Arthur_Officiel bravo aux équipes de @Satisfaction_tv 👏👏 #VTEP #TF1
RT @08_rosemary03: La magie de #VTEP avec @Arthur_Officiel et tous les invités et surtout @CartmanOfficiel et @cyrilferaud ❤❤❤❤❤❤
RT @mauritiusgirly: Trop balaise la piece ou ont se retrouve a lenvers 🤣🤣 #VTEP
RT @Rodrigu64581700: L'accident ça nous fera un peu le buzz 😂😂😂 @Arthur_Officiel t'es un enfoiré 😂😂😂 #VTEP
RT @chriiistine__75: Ah oui, de nouveau le plateau a 360 e

In [7]:
status_list = []

In [8]:
for status in tweepy.Cursor(api.home_timeline).items(10):
    # Process a single status
    status_list.append(status._json)

Define a streaming listener

In [10]:
class MyListener(StreamListener):
 
    def on_data(self, data):
        try:
            with open('python.json', 'a') as f:
                f.write(data)
                return True
        except BaseException as e:
            print("Error on_data: %s" % str(e))
        return True
 
    def on_error(self, status):
        print(status)
        return True

Launch a real time listener

twitter_stream = Stream(auth, MyListener())
twitter_stream.filter(track=['#python'])

## Part 2: Text Pre-processing

Create a tokenizer

In [13]:
emoticons_str = r"""
    (?:
        [:=;] # Eyes
        [oO\-]? # Nose (optional)
        [D\)\]\(\]/\\OpP] # Mouth
    )"""
 
regex_str = [
    emoticons_str,
    r'<[^>]+>', # HTML tags
    r'(?:@[\w_]+)', # @-mentions
    r"(?:\#+[\w_]+[\w\'_\-]*[\w_]+)", # hash-tags
    r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', # URLs
 
    r'(?:(?:\d+,?)+(?:\.?\d+)?)', # numbers
    r"(?:[a-z][a-z'\-_]+[a-z])", # words with - and '
    r'(?:[\w_]+)', # other words
    r'(?:\S)' # anything else
]
    

In [14]:
tokens_re = re.compile(r'('+'|'.join(regex_str)+')', re.VERBOSE | re.IGNORECASE)
emoticon_re = re.compile(r'^'+emoticons_str+'$', re.VERBOSE | re.IGNORECASE)

In [15]:
def tokenize(s):
    return tokens_re.findall(s)
 
def preprocess(s, lowercase=False):
    tokens = tokenize(s)
    if lowercase:
        tokens = [token if emoticon_re.search(token) else token.lower() for token in tokens]
    return tokens

In [16]:
tweet = 'RT @marcobonzanini: just an example! :D http://example.com #NLP'
print(preprocess(tweet))

['RT', '@marcobonzanini', ':', 'just', 'an', 'example', '!', ':D', 'http://example.com', '#NLP']


In [22]:
tokens_list = []

In [26]:
for status in status_list:
    tokens = preprocess(status['text'])
    tokens_list.append(tokens)

In [28]:
tokens_list[0]

['RT',
 '@rocmaelias',
 ':',
 'Jvien',
 'de',
 'découvrir',
 'que',
 'le',
 'ptit',
 'truc',
 'jaune',
 "c'est",
 'comme',
 'le',
 'jaune',
 'de',
 "l'oeuf",
 'https://t.co/UNDANnNc6y']

## Part 3: Term Frequencies

Importing stop-words

In [48]:
punctuation = list(string.punctuation)
stop = stopwords.words('french') + punctuation + ['RT']

Count term occurences 

In [60]:
count_all = Counter()
for status in status_list:
    
    # Create a list with all the terms
    terms_stop = [term for term in preprocess(status['text']) if term not in stop and not term.startswith(('#', '@'))]
    
    terms_hash = [term for term in preprocess(status['text']) if term.startswith('#')]
    
    # Update the counter
    count_all.update(terms_stop)
    
    # Print the first 5 most frequent words
    print(count_all.most_common(5))

[('jaune', 2), ('Jvien', 1), ('découvrir', 1), ('ptit', 1), ('truc', 1)]
[('jaune', 2), ('Jvien', 1), ('découvrir', 1), ('ptit', 1), ('truc', 1)]
[('jaune', 2), ('Jvien', 1), ('découvrir', 1), ('ptit', 1), ('truc', 1)]
[('jaune', 2), ('veux', 2), ('Jvien', 1), ('découvrir', 1), ('ptit', 1)]
[('jaune', 2), ('veux', 2), ('Jvien', 1), ('découvrir', 1), ('ptit', 1)]
[('😅', 3), ('jaune', 2), ('veux', 2), ('nouvelles', 2), ('Jvien', 1)]
[('😅', 3), ('jaune', 2), ("c'est", 2), ('Je', 2), ('veux', 2)]
[('❤', 6), ('😅', 3), ('jaune', 2), ("c'est", 2), ('Je', 2)]
[('❤', 6), ('🤣', 3), ('😅', 3), ('jaune', 2), ("c'est", 2)]
[('❤', 6), ('😂', 6), ('🤣', 3), ('😅', 3), ('jaune', 2)]


In [62]:
terms_bigram = bigrams(terms_stop)

In [66]:
print(list(terms_bigram))

[("L'accident", 'ça'), ('ça', 'fera'), ('fera', 'peu'), ('peu', 'buzz'), ('buzz', '😂'), ('😂', '😂'), ('😂', '😂'), ('😂', "t'es"), ("t'es", 'enfoir'), ('enfoir', 'é'), ('é', '😂'), ('😂', '😂'), ('😂', '😂')]


## Part 4: Term Co-occurrences

In this part, we will stream recents tweets about this last European Champion's Leagues matchs played this week.
So, will use the stream function to stream the #UCL .

Launch a real time listener on #UCL

In [108]:
query = '#UCL'
max_tweets = 1000

In [109]:
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(max_tweets)]

save tweets in a json file

In [112]:
with open('ucl_tweets.json', 'w') as outfile:
    for status in searched_tweets:
        json.dump(status._json, outfile)

searched_tweets[0]._json

Import english stop word and punctuation

In [118]:
punctuation = list(string.punctuation)
stop = stopwords.words('french') + punctuation + ['RT', '️', '…', '', ] + stopwords.words('english')

Count term occurences 

In [141]:
count_all = Counter()
for status in searched_tweets:
    
    # Create a list with all the terms
    terms_stop = [term for term in preprocess(status._json['text']) if term not in stop and not term.startswith(('#', '@')) and not term.isdigit()]
    
    # Update the counter
    count_all.update(terms_stop)

In [144]:
# Print the most frequent words
print(count_all.most_common(40))

[('Messi', 328), ('⚽', 304), ('🏆', 243), ('Week', 155), ('Goal', 135), ('Sterling', 130), ('Lionel', 126), ('David', 126), ('Neres', 125), ('Raheem', 124), ('Philippe', 124), ('Coutinho', 124), ('🤔', 117), ('Agree', 117), ('ord', 117), ('🥇', 98), ('🗓', 93), ('Majestic', 92), ('scoops', 92), ('prize', 92), ('https://t.co/L2qzAL39wg', 92), ('😂', 88), ('\U0001f929', 84), ('us', 81), ('’', 81), ('If', 80), ('nowt', 78), ('else', 78), ('given', 78), ('priceless', 78), ('moments', 78), ('How', 78), ('https://t.co/pChlaUsJG4', 78), ('Redondo', 77), ('😍', 76), ('amp', 68), ('best', 65), ('assists', 64), ('history', 64), ('🔥', 64)]


In [125]:
count_all = Counter()
for status in searched_tweets:
    
    # Create a list with all the terms
    terms_hash = [term for term in preprocess(status._json['text']) if term not in stop and term.startswith(('#')) and not term.isdigit()]
    
    # Update the counter
    count_all.update(terms_hash)

In [126]:
# Print the most frequent hashtags
print(count_all.most_common(40))

[('#UCL', 696), ('#POTW', 99), ('#OnThisDay', 85), ('#VAR', 80), ('#FlashbackFriday', 51), ('#COYS', 40), ('#GOTW', 23), ('#ajabay', 20), ('#LFC', 19), ('#FCPorto', 17), ('#SomosPorto', 17), ('#Ajax', 17), ('#TalDíaComoHoy', 16), ('#juvaja', 11), ('#MCITOT', 11), ('#ウイイレ', 8), ('#ウイイレアプリ', 8), ('#ChampionsLeagu', 8), ('#ucl', 8), ('#UEL', 6), ('#MUFC', 6), ('#JuveAjax', 6), ('#OTD', 4), ('#ChampionsLeague', 4), ('#TICKETMANIA', 4), ('#Datos', 4), ('#ChampionsLea', 4), ('#Messi', 3), ('#FIFA19', 3), ('#Škriniar', 2), ('#Eintracht', 2), ('#totaja', 2), ('#LdC', 2), ('#YNWA', 2), ('#GETREADY', 2), ('#LaLiga', 2), ('#DrakeCurse', 2), ('#barcelona', 2), ('#Guardiola', 2), ('#ajajuv', 2)]


Display Term co-occurrences

In [133]:
com = defaultdict(lambda : defaultdict(int))
 
# f is the file pointer to the JSON data set
for status in searched_tweets: 
    
    terms_only = [term for term in preprocess(status._json['text']) 
                  if term not in stop 
                  and not term.startswith(('#', '@'))
                  and not term.isdigit()]
 
    # Build co-occurrence matrix
    for i in range(len(terms_only)-1):            
        for j in range(i+1, len(terms_only)):
            w1, w2 = sorted([terms_only[i], terms_only[j]])                
            if w1 != w2:
                com[w1][w2] += 1

In [134]:
com_max = []
# For each term, look for the most common co-occurrent terms
for t1 in com:
    t1_max_terms = sorted(com[t1].items(), key=operator.itemgetter(1), reverse=True)[:5]
    for t2, t2_count in t1_max_terms:
        com_max.append(((t1, t2), t2_count))

In [135]:
# Get the most frequent co-occurrences
terms_max = sorted(com_max, key=operator.itemgetter(1), reverse=True)
print(terms_max[:5])

[(('⚽', '🏆'), 472), (('Messi', '⚽'), 271), (('⚽', '🤔'), 234), (('Goal', '⚽'), 234), (('Goal', '🏆'), 234)]


Calculate a term co-occurence

In [136]:
search_word = "Messi"

In [138]:
count_search = Counter()
for status in searched_tweets:
    terms_only = [term for term in preprocess(status._json['text']) 
                  if term not in stop 
                  and not term.startswith(('#', '@'))
                  and not term.isdigit()]
    if search_word in terms_only:
        count_search.update(terms_only)

In [139]:
print("Co-occurrence for %s:" % search_word)
print(count_search.most_common(20))

Co-occurrence for Messi:
[('Messi', 328), ('⚽', 271), ('🏆', 234), ('Week', 152), ('Goal', 135), ('Lionel', 126), ('Raheem', 124), ('Sterling', 124), ('David', 124), ('Neres', 124), ('Philippe', 124), ('Coutinho', 124), ('🤔', 117), ('Agree', 117), ('ord', 117), ('🥇', 98), ('Majestic', 92), ('scoops', 92), ('prize', 92), ('https://t.co/L2qzAL39wg', 92)]


## Part 5: Data Visualisation Basics

In [150]:
most_common_words = count_all.most_common(15)

In [151]:
labels, freq = zip(*most_common_words)
data = {'data': freq, 'x': labels}
bar = vincent.Bar(data, iter_idx='x')
bar.to_json('term_freq.json')

In [147]:
bar.to_json('term_freq.json', html_out=True, html_path='chart.html')

### Deal with time-series

In [153]:
dates_UCL = []

for status in searched_tweets:
    
    # let's focus on hashtags only at the moment
    terms_hash = [term for term in preprocess(status._json['text']) if term.startswith('#')]
    # track when the hashtag is mentioned
    if '#UCL' in terms_hash:
        dates_UCL.append(status._json['created_at'])
 

In [154]:
# a list of "1" to count the hashtags
ones = [1]*len(dates_UCL)

In [156]:
len(ones) 

696

In [165]:
# the index of the series
idx = pd.DatetimeIndex(dates_UCL)
# the actual series (at series of 1s for the moment)
UCL = pd.Series(ones, index=idx)

In [166]:
# Resampling / bucketing
per_minute = UCL.resample('1Min').sum().fillna(0)

Plot the time series on a graph

In [170]:
time_chart = vincent.Line(per_minute)
time_chart.axis_titles(x='Time', y='Freq')
time_chart.to_json('time_chart.json')

Gather time-seres data from other hashtags

In [177]:
# a function to get timeSeries data on 1 hastag given in parameter,
#  the other parameter is the tweet's source ( a tweepy's cursor collection)
# return a dataframe with time-series (grouped by minutes) as index
def get_time_series(hastag, searched_tweets):
    
    dates = []

    for status in searched_tweets:
    
        # let's focus on hashtags only at the moment
        terms_hash = [term for term in preprocess(status._json['text'])]
        # track when the hashtag is mentioned
        if hastag in terms_hash:
            dates.append(status._json['created_at'])
            
    # a list of "1" to count the hashtags
    ones = [1]*len(dates)
    
    # the index of the series
    idx = pd.DatetimeIndex(dates)
    # the actual series (at series of 1s for the moment)
    df_hashtag = pd.Series(ones, index=idx)
    
    # Resampling / bucketing
    per_minute = df_hashtag.resample('1Min').sum().fillna(0)
    
    return per_minute
    

Get time-Series data for 'Messi', 'Coutinho' and '⚽'

In [186]:
per_minute_messi = get_time_series('Messi', searched_tweets)
per_minute_cou = get_time_series('Coutinho', searched_tweets)
per_minute_ball = get_time_series('⚽', searched_tweets)


Plot all these data on the same graph

In [187]:
# all the data together
tweets_data = dict(MESSI=per_minute_messi, COU=per_minute_cou, BALL=per_minute_ball)
# we need a DataFrame, to accommodate multiple series
all_matches = pd.DataFrame(data=tweets_data,
                               index=per_minute_messi.index)

In [188]:
# Resampling as above
#all_matches = all_matches.resample('1Min').sum().fillna(0)
 
# and now the plotting
time_chart = vincent.Line(all_matches[['COU', 'MESSI']])
time_chart.axis_titles(x='Time', y='Freq')
time_chart.legend(title='Matches')
time_chart.to_json('multi_times_chart.json')

In [144]:
# Print the most frequent words
print(count_all.most_common(40))

[('Messi', 328), ('⚽', 304), ('🏆', 243), ('Week', 155), ('Goal', 135), ('Sterling', 130), ('Lionel', 126), ('David', 126), ('Neres', 125), ('Raheem', 124), ('Philippe', 124), ('Coutinho', 124), ('🤔', 117), ('Agree', 117), ('ord', 117), ('🥇', 98), ('🗓', 93), ('Majestic', 92), ('scoops', 92), ('prize', 92), ('https://t.co/L2qzAL39wg', 92), ('😂', 88), ('\U0001f929', 84), ('us', 81), ('’', 81), ('If', 80), ('nowt', 78), ('else', 78), ('given', 78), ('priceless', 78), ('moments', 78), ('How', 78), ('https://t.co/pChlaUsJG4', 78), ('Redondo', 77), ('😍', 76), ('amp', 68), ('best', 65), ('assists', 64), ('history', 64), ('🔥', 64)]
